In [51]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [52]:
df1 = pd.read_csv("v1-start.csv")
df1.head()

,type,external_id,name,winery,size,internal_notes,purchase_price,VKP,sales_price,Enoteca,takeaway_price
0,SPARKLING,10110.0,"Prosecco di Valdobbiadene, Extra Brut",Bortolin,NaN,Holzer,8.00 €,23.76 €,35.00 €,13.00 €,13.00
1,SPARKLING,10116.0,Arunda Brut Rosè Exellor,Arunda,NaN,Karadar,25.00 €,53.68 €,54.00 €,39.65 €,39.50
2,SPARKLING,10117.0,Hausmannhof Reserve 2013,Haderburg,NaN,direkt,35.00 €,71.28 €,69.00 €,55.51 €,55.50
3,SPARKLING,10118.0,Solera Extra Brut,Marco Buvoli,NaN,NaN,29.00 €,60.72 €,75.00 €,45.99 €,46.00
4,SPARKLING,10119.0,Haderburg Spumante Brut,Haderburg,NaN,direkt,16.00 €,37.84 €,39.00 €,28.60 €,28.60


# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [53]:
df1.head(2)

,type,external_id,name,winery,size,internal_notes,purchase_price,VKP,sales_price,Enoteca,takeaway_price
0,SPARKLING,10110.0,"Prosecco di Valdobbiadene, Extra Brut",Bortolin,NaN,Holzer,8.00 €,23.76 €,35.00 €,13.00 €,13.00
1,SPARKLING,10116.0,Arunda Brut Rosè Exellor,Arunda,NaN,Karadar,25.00 €,53.68 €,54.00 €,39.65 €,39.50


In [54]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [55]:
import re

# copy values from original columns to new columns
df['external_id'] = df1['external_id']
df['type'] = df1['type']
df['name'] = df1['name']
df['winery_name'] = df1['winery']
df["vintage"] = df1["name"].apply(lambda x: int(re.search(r'\d{4}', x).group()) if re.search(r'\d{4}', x) else None) # extrapolate from name, where i match regex for 4 digits
df["quantity"] = 50
df["internal_notes"] = df1["internal_notes"]
df["visible"] = True

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,10110.0,SPARKLING,"Prosecco di Valdobbiadene, Extra Brut",Bortolin,NaN,NaN,NaN,NaN,NaN,50,NaN,Holzer,True
1,10116.0,SPARKLING,Arunda Brut Rosè Exellor,Arunda,NaN,NaN,NaN,NaN,NaN,50,NaN,Karadar,True
2,10117.0,SPARKLING,Hausmannhof Reserve 2013,Haderburg,NaN,NaN,2013.0,NaN,NaN,50,NaN,direkt,True
3,10118.0,SPARKLING,Solera Extra Brut,Marco Buvoli,NaN,NaN,NaN,NaN,NaN,50,NaN,NaN,True
4,10119.0,SPARKLING,Haderburg Spumante Brut,Haderburg,NaN,NaN,NaN,NaN,NaN,50,NaN,direkt,True


In [56]:
# size
df['size'] = df1["size"].fillna("BOTTLE")

# convert prices to cents
# df['price'] = df1['sales_price']
# fill empty prices with 0
df1['purchase_price'] = df1['purchase_price'].fillna(0)
df1['sales_price'] = df1['sales_price'].fillna(0)
df1['takeaway_price'] = df1['takeaway_price'].fillna(0)

# convert prices to cents
df['purchase_price'] = df1['purchase_price'].apply(lambda x: int(float(str(x).replace('€','').strip())*100)) # convert to cents
df['price'] = df1['sales_price'].apply(lambda x: int(float(str(x).replace('€','').strip())*100)) # convert to cents
df['takeaway_price'] = df1['takeaway_price'].apply(lambda x: int(float(str(x).replace('€','').strip())*100)) # convert to cents

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible,takeaway_price
0,10110.0,SPARKLING,"Prosecco di Valdobbiadene, Extra Brut",Bortolin,NaN,BOTTLE,NaN,3500,800,50,NaN,Holzer,True,1300
1,10116.0,SPARKLING,Arunda Brut Rosè Exellor,Arunda,NaN,BOTTLE,NaN,5400,2500,50,NaN,Karadar,True,3950
2,10117.0,SPARKLING,Hausmannhof Reserve 2013,Haderburg,NaN,BOTTLE,2013.0,6900,3500,50,NaN,direkt,True,5550
3,10118.0,SPARKLING,Solera Extra Brut,Marco Buvoli,NaN,BOTTLE,NaN,7500,2900,50,NaN,NaN,True,4600
4,10119.0,SPARKLING,Haderburg Spumante Brut,Haderburg,NaN,BOTTLE,NaN,3900,1600,50,NaN,direkt,True,2860


In [57]:
df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible,takeaway_price
0,10110.0,SPARKLING,"Prosecco di Valdobbiadene, Extra Brut",Bortolin,,BOTTLE,0,3500,800,50,,Holzer,True,1300
1,10116.0,SPARKLING,Arunda Brut Rosè Exellor,Arunda,,BOTTLE,0,5400,2500,50,,Karadar,True,3950
2,10117.0,SPARKLING,Hausmannhof Reserve 2013,Haderburg,,BOTTLE,2013,6900,3500,50,,direkt,True,5550
3,10118.0,SPARKLING,Solera Extra Brut,Marco Buvoli,,BOTTLE,0,7500,2900,50,,,True,4600
4,10119.0,SPARKLING,Haderburg Spumante Brut,Haderburg,,BOTTLE,0,3900,1600,50,,direkt,True,2860


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [58]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False